In [1]:
import pandas
import os
import json

In [2]:
with open(os.environ['DATADIR']+'/mfp/days.json') as f:
    days = json.load(f)

In [3]:
daily_cals = pandas.json_normalize(days)
daily_cals['Date'] = pandas.to_datetime(daily_cals['Date'])

In [4]:
renpho = pandas.read_csv(os.environ['DATADIR'] + '/renpho/renpho_health_merge.csv')
renpho['Date'] = pandas.to_datetime(renpho['Date'])

In [26]:
renpho = renpho.merge(daily_cals, left_on='Date',right_on='Date')
renpho.to_csv(os.environ['DATADIR'] + '/joined/fullhealth.csv')

In [6]:
renpho.columns.values

array(['Time of Measurement', 'Weight', 'BMI', 'Body Fat',
       'Fat-free Body Weight', 'Subcutaneous Fat', 'Visceral Fat',
       'Body Water', 'Skeletal Muscle', 'Muscle Mass', 'Bone Mass',
       'Protein', 'BMR', 'Metabolic Age', 'Date', 'Time', 'Start',
       'Finish', 'Active Calories (kcal)', 'Body Fat Percentage (%)',
       'Body Mass Index (count)', 'Distance (mi)',
       'Flights Climbed (count)', 'Resting Calories (kcal)',
       'Steps (count)', 'Weight (lb)', 'Breakfast.calories',
       'Breakfast.carbohydrates', 'Breakfast.fat', 'Breakfast.protein',
       'Breakfast.sodium', 'Breakfast.sugar', 'Lunch.calories',
       'Lunch.carbohydrates', 'Lunch.fat', 'Lunch.protein',
       'Lunch.sodium', 'Lunch.sugar', 'Dinner.calories',
       'Dinner.carbohydrates', 'Dinner.fat', 'Dinner.protein',
       'Dinner.sodium', 'Dinner.sugar', 'Snacks.calories',
       'Snacks.carbohydrates', 'Snacks.fat', 'Snacks.protein',
       'Snacks.sodium', 'Snacks.sugar'], dtype=object)

In [21]:
select_features = pandas.DataFrame()
select_features['date'] = renpho['Date']
select_features['weight'] = renpho['Weight'].apply(lambda x: float(x.replace("Lb","")))
select_features['cal_out'] = renpho['Active Calories (kcal)'] + renpho['Resting Calories (kcal)']
select_features['cal_in'] = renpho['Breakfast.calories'].fillna(0) + renpho['Lunch.calories'].fillna(0) + renpho['Dinner.calories'].fillna(0) + renpho['Snacks.calories'].fillna(0)
select_features['carbohydrates'] = renpho['Breakfast.carbohydrates'].fillna(0) + renpho['Lunch.carbohydrates'].fillna(0) + renpho['Dinner.carbohydrates'].fillna(0) + renpho['Snacks.carbohydrates'].fillna(0)
select_features['protein'] = renpho['Breakfast.protein'].fillna(0) + renpho['Lunch.protein'].fillna(0) + renpho['Dinner.protein'].fillna(0) + renpho['Snacks.protein'].fillna(0)
select_features['fat'] = renpho['Breakfast.fat'].fillna(0) + renpho['Lunch.fat'].fillna(0) + renpho['Dinner.fat'].fillna(0) + renpho['Snacks.fat'].fillna(0)

select_features['cal_net'] = select_features['cal_in'] - select_features['cal_out']



In [22]:
select_features.describe()

,weight,cal_out,cal_in,carbohydrates,protein,fat,cal_net
count,490.000000,490.000000,490.000000,490.000000,490.000000,490.000000,490.000000
mean,201.879184,3325.541849,3283.169388,320.977551,145.508163,146.595918,-42.372461
std,2.907591,706.707196,1146.655946,130.290981,50.505592,55.140386,1345.240022
min,193.800000,228.460279,0.000000,0.000000,0.000000,0.000000,-4100.221144
25%,199.850000,2967.287880,2457.000000,232.250000,107.250000,107.250000,-962.390184
50%,202.000000,3386.491831,3192.000000,308.000000,144.000000,142.500000,-93.514807
75%,203.750000,3676.570026,4060.000000,396.250000,181.000000,182.000000,737.000000
max,212.000000,5287.163236,6760.000000,803.000000,278.000000,310.000000,4232.441442


In [23]:
select_features.corr()

,weight,cal_out,cal_in,carbohydrates,protein,fat,cal_net
weight,1.000000,-0.156813,-0.059677,-0.009813,-0.092956,-0.094663,0.031513
cal_out,-0.156813,1.000000,0.002829,-0.015850,0.036053,0.010254,-0.522928
cal_in,-0.059677,0.002829,1.000000,0.896460,0.716115,0.880938,0.850894
carbohydrates,-0.009813,-0.015850,0.896460,1.000000,0.551675,0.673575,0.772451
protein,-0.092956,0.036053,0.716115,0.551675,1.000000,0.674130,0.591462
fat,-0.094663,0.010254,0.880938,0.673575,0.674130,1.000000,0.745508
cal_net,0.031513,-0.522928,0.850894,0.772451,0.591462,0.745508,1.000000
